# Transformer的PyTorch从头实现

此项目使用了PyTorch，请参考以下页面安装PyTorch：
[https://pytorch.org/get-started/locally/](https://pytorch.org/get-started/locally/)

除了PyTorch外，此项目还需要以下Python库：
* torchdata
* torchtext

可以使用以下pip命令安装它们：
```
pip install torchtext torchdata
```

此项目使用了spaCy tokenizer，需要使用以下命令安装spaCy语言包de_core_news_sm和en_core_web_sm：
```
python -m spacy download de_core_news_sm
python -m spacy download en_core_web_sm
```

## 0. 依赖和常量

### 0.1 依赖

In [1]:
# Python imports.
from enum import Enum

# Third-party imports.
import torchtext

### 0.2 常量

In [2]:
# Define dataset types.
class Dataset(Enum):
    MULTI30K = 1

# Define which dataset to use.
DATASET = Dataset.MULTI30K

## 1. 数据准备

我们使用multi30k数据集。multi30k数据集是WMT16数据集的一部分，31,014条英语和德语句子以及对应的图片。但我们只使用其中的英语和德语句子。multi30k数据集的详细介绍请参考[https://www.statmt.org/wmt16/multimodal-task.html#task1](https://www.statmt.org/wmt16/multimodal-task.html#task1)

training、dev和test数据集各自的规模如下：
* training：29,000
* dev: 1014
* test: 1000

### 1.1 辅助函数定义

函数display_samples打印出头部几条数据：

In [12]:
def display_samples(dataset, num):
    iterator = iter(dataset)
    try:
        for i in range(num):
            print(f'{i+1}: {next(iterator)}')
    except StopIteration:
        print(f'[Error] Size of dataset is smaller than {num}.')

### 1.2 数据加载

加载training、dev和test数据集。这些数据集的数据类型为torch.utils.data.datapipes.iter.grouping.ShardingFilterIterDataPipe。

In [15]:
training, dev, text = torchtext.datasets.Multi30k()

In [16]:
display_samples(training, 5)

1: ('Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.', 'Two young, White males are outside near many bushes.')
2: ('Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem.', 'Several men in hard hats are operating a giant pulley system.')
3: ('Ein kleines Mädchen klettert in ein Spielhaus aus Holz.', 'A little girl climbing into a wooden playhouse.')
4: ('Ein Mann in einem blauen Hemd steht auf einer Leiter und putzt ein Fenster.', 'A man in a blue shirt is standing on a ladder cleaning a window.')
5: ('Zwei Männer stehen am Herd und bereiten Essen zu.', 'Two men are at the stove preparing food.')


### 1.3 数据tokenization

In [6]:
tokenizer_de = torchtext.data.get_tokenizer('spacy', language='de_core_news_sm')
tokenizer_en = torchtext.data.get_tokenizer('spacy', language='en_core_web_sm')


In [7]:
iter_train = iter(train)
sentence = next(iter_train)
sentence_de = sentence[0]
sentence_en = sentence[1]
print(sentence_de + ' -> ' + str(tokenizer_de(sentence_de)))
print(tokenizer_en('I don\'t have retreat.'))

Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche. -> ['Zwei', 'junge', 'weiße', 'Männer', 'sind', 'im', 'Freien', 'in', 'der', 'Nähe', 'vieler', 'Büsche', '.']
['I', 'do', "n't", 'have', 'retreat', '.']
